In [ ]:
!pip install neuralforecast

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown

import matplotlib.pyplot as plt
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS
from neuralforecast.utils import AirPassengersDF

In [ ]:
# Split data:
ydf = AirPassengersDF

ytrain_df = ydf[ydf.ds <= '1959-12-31'] # 132 timepoints to train
ytest_df = ydf[ydf.ds > '1959-12-31'] # 12 timepoints to test

In [ ]:
# Fit and predict with N-BEATS and N-HiTS models:
horizon = len(ytest_df)

models = [
  NBEATS(input_size=2 * horizon, h=horizon, max_epochs=50),
  NHITS(input_size=2 * horizon, h=horizon, max_epochs=50)
]

nforecast = NeuralForecast(models=models, freq='M')
nforecast.fit(df=ytrain_df)

yhat_df = nforecast.predict().reset_index()

In [ ]:
# Plot predictions:
fig, ax = plt.subplots(1, 1, figsize = (20, 7))

yhat_df = ytest_df.merge(yhat_df, how='left', on=['unique_id', 'ds'])
plot_df = pd.concat([ytrain_df, yhat_df]).set_index('ds')

plot_df[['y', 'NBEATS', 'NHITS']].plot(ax=ax, linewidth=2)

ax.set_title('AirPassengers Forecast', fontsize=22)
ax.set_ylabel('Monthly Passengers', fontsize=20)
ax.set_xlabel('Timestamp [t]', fontsize=20)
ax.legend(prop={'size': 15})
ax.grid()